# 浪子之心科技
# 此代码仅为演示wav2lip推理结束后用CodeFormer进行高清化的过程。
# 效果比Wav2Lip-GFPGAN好。


In [1]:
#@title 一、下载wav2lip源码和权重并安装wav2lip环境
#@title
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/zabique/Wav2Lip

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install -q youtube-dl
!pip install ffmpeg-python
!pip install librosa==0.9.1

#this code for recording audio
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

# from IPython.display import clear_output
# clear_output()
print("\nDone")

Cloning into 'Wav2Lip'...
remote: Enumerating objects: 378, done.
remote: Total 378 (delta 0), reused 0 (delta 0), pack-reused 378 (from 1)
Receiving objects: 100% (378/378), 526.98 KiB | 6.51 MiB/s, done.
Resolving deltas: 100% (205/205), done.
--2024-10-29 03:10:27--  https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA
Resolving iiitaphyd-my.sharepoint.com (iiitaphyd-my.sharepoint.com)... 52.105.227.53, 2a01:111:f402:f0c9::53
Connecting to iiitaphyd-my.sharepoint.com (iiitaphyd-my.sharepoint.com)|52.105.227.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 435801865 (416M) [application/octet-stream]
Saving to: ‘/content/Wav2Lip/checkpoints/wav2lip_gan.pth’

/content/Wav2Lip/ch 100%[===================>] 415.61M  34.7MB/s    in 13s     

2024-10-29 03:10:40 (32.5 MB/s) - ‘/content/Wav2Lip/checkpoints/wav2lip_gan.pth’ saved [435801865/435801865]

--2024-10-2

In [2]:
#@title 二、下载CodeFormer的源码权重并安装依赖

# 下载CodeFormer的源码
%cd /content
!rm -rf CodeFormer
!git clone https://github.com/sczhou/CodeFormer.git
%cd /content/CodeFormer

/content
Cloning into 'CodeFormer'...
remote: Enumerating objects: 614, done.
remote: Counting objects: 100% (298/298), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 614 (delta 204), reused 184 (delta 178), pack-reused 316 (from 1)
Receiving objects: 100% (614/614), 17.31 MiB | 17.18 MiB/s, done.
Resolving deltas: 100% (297/297), done.
/content/CodeFormer


In [3]:
# 下载下载CodeFormer的权重
%cd /content/CodeFormer
!python scripts/download_pretrained_models.py facelib
!python scripts/download_pretrained_models.py CodeFormer

/content/CodeFormer
Traceback (most recent call last):
  File "/content/CodeFormer/scripts/download_pretrained_models.py", line 5, in <module>
    from basicsr.utils.download_util import load_file_from_url
ModuleNotFoundError: No module named 'basicsr'
Traceback (most recent call last):
  File "/content/CodeFormer/scripts/download_pretrained_models.py", line 5, in <module>
    from basicsr.utils.download_util import load_file_from_url
ModuleNotFoundError: No module named 'basicsr'


In [ ]:
# 安装CodeFormer的依赖
%cd /content/CodeFormer
!python basicsr/setup.py develop

/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
 

In [5]:
#@title 三、上传视频和音频
# 创建相关的文件夹
%cd /content
!mkdir output
!mkdir input
# 上传音视频
%cd /content/input
from google.colab import files
from io import BytesIO
uploaded = files.upload()

/content
mkdir: cannot create directory ‘output’: File exists
mkdir: cannot create directory ‘input’: File exists
/content/input


Saving basic_vedio.mp4 to basic_vedio.mp4


In [6]:
#@title 四、用wav2lip初步生成修改嘴型后的视频

# 先清理之前生成的临时文件
%cd /content
!rm -rf output
!mkdir output
!rm -rf /content/Wav2Lip/temp/result.avi
!rm -rf /content/Wav2Lip/temp/temp.wav
!rm -rf /content/Wav2Lip/results/result_voice.mp4
!rm -rf images
!mkdir images
%cd /content/CodeFormer
!rm -rf results
!mkdir results

/content
/content/CodeFormer


In [7]:
# 用wav2lip进行推理
# checkpoints 可以用wav2lip.pth 或者wav2lip_gan.pt
%cd /content/Wav2Lip
!python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face '/content/input/basic_vedio.mp4' --audio '/content/input/A_10s.wav'

/content/Wav2Lip
Using cuda for inference.
Reading video frames...
Number of frames available for inference: 281
/content/Wav2Lip/audio.py:100: FutureWarning: Pass sr=16000, n_fft=800 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.filters.mel(hp.sample_rate, hp.n_fft, n_mels=hp.num_mels,
(80, 787)
Length of mel chunks: 243
  0% 0/2 [00:00<?, ?it/s]/content/Wav2Lip/face_detection/detection/sfd/sfd_detector.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects 

In [8]:
# 将Wav2Lip生成的视频转化成图片保存到images
%cd /content
!mkdir images
%cd /content/Wav2Lip
import cv2
from tqdm import tqdm
from os import path
import os

vidcap = cv2.VideoCapture("/content/Wav2Lip/results/result_voice.mp4")
numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", numberOfFrames)

for frameNumber in tqdm(range(numberOfFrames)):
    _,image = vidcap.read()
    cv2.imwrite(path.join('/content/images', str(frameNumber).zfill(4)+'.jpg'), image)

/content
mkdir: cannot create directory ‘images’: File exists
/content/Wav2Lip
FPS:  25.0 Frames:  243


100%|██████████| 243/243 [00:01<00:00, 146.65it/s]


In [9]:
#@title 五、用CodeFormer进行人脸修复

%cd /content/CodeFormer
# 定义相关函数
import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1)
  plt.title('Input', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('CodeFormer', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

/content/CodeFormer


In [11]:
!pip install lpips

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.4 MB/s eta 0:00:00


In [14]:
!pip install basicsr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 12.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 21.8 MB/s eta 0:00:00
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214817 sha256=e457abb2d243bc9f84540fcf5f157c152e81f1c55b2e2ffabe641eb03c42f30b
  Stored in directory: /root/.cache/pip/wheels/38/83/99/2d8437cc652a01af27df5ff037a4075e95b52d67705c5f30ca
Successfully built basicsr


In [16]:
# 用CodeFormer进行推理
%cd /content/CodeFormer
## 只修复人脸
# CODEFORMER_FIDELITY = 0.5
# !python inference_codeformer.py -w $CODEFORMER_FIDELITY --has_aligned --input_path /content/images

# whole images ##整体图片修复
CODEFORMER_FIDELITY = 0.7
!python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path /content/images --bg_upsampler realesrgan

/content/CodeFormer
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/RealESRGAN_x2plus.pth" to /content/CodeFormer/weights/realesrgan/RealESRGAN_x2plus.pth

100% 64.0M/64.0M [00:00<00:00, 342MB/s]
/content/CodeFormer/basicsr/utils/realesrgan_utils.py:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` fo

In [17]:
# 生成修复后的新视频
import os
restoredFramesPath = '/content/CodeFormer/results/images_0.7' + '/final_results/'
processedVideoOutputPath = '/content/output'

dir_list = os.listdir(restoredFramesPath)
dir_list.sort()

import cv2
import numpy as np

batch = 0
batchSize = 600
from tqdm import tqdm
for i in tqdm(range(0, len(dir_list), batchSize)):
  img_array = []
  start, end = i, i+batchSize
  print("processing ", start, end)
  for filename in  tqdm(dir_list[start:end]):
      filename = restoredFramesPath+filename;
      img = cv2.imread(filename)
      if img is None:
        continue
      height, width, layers = img.shape
      size = (width,height)
      img_array.append(img)


  out = cv2.VideoWriter(processedVideoOutputPath+'/batch_'+str(batch).zfill(4)+'.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
  batch = batch + 1

  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release()

# 音视频合并 Generate audio video
audio = ffmpeg.input(f'/content/input/A_10s.wav')
video = ffmpeg.input(f'/content/output/batch_0000.avi')
print("合并视音频")
out = ffmpeg.output(video, audio, f'/content/output/final.mp4')
out.run()
print("恭喜您，音视频合并完成，存放在output/final.mp4")

  0%|          | 0/1 [00:00<?, ?it/s]

processing  0 600



100%|██████████| 1/1 [00:29<00:00, 29.87s/it]


合并视音频


Error: ffmpeg error (see stderr output for detail)